# MultiBand Single Pass (MBSP) DemoThis notebook demonstrates the use of the `mbsp` module for methane plume analysis.

## Setup
Import the required packages and authenticate Earth Engine.

In [ ]:
import eeimport geemapimport matplotlib.pyplot as pltimport numpy as npimport sys, pathlibsys.path.append(str(pathlib.Path.cwd()/"src"))import mbspee.Initialize()

## Processing functions

In [ ]:
def analyze_scene(lat, lon, start_date, end_date):
    col = mbsp.load_l1c_collection(lat, lon, start_date, end_date)
    img = col.first().resample('bicubic').reproject('EPSG:32633', scale=20)
    b11 = geemap.ee_to_numpy(img.select('B11'))
    b12 = geemap.ee_to_numpy(img.select('B12'))
    mbsp_arr, _ = mbsp.mbsp_arrays(b11, b12)
    column = mbsp.detrend_and_smooth(mbsp.mbsp_to_column(mbsp_arr))
    mask = mbsp.plume_mask(column)
    rate = mbsp.ime_and_rate(column, mask, wind_speed=5)
    return column, mask, rate

def show_results(column, mask, rate):
    plt.figure(figsize=(6, 6))
    plt.imshow(column, cmap='viridis')
    plt.contour(mask, colors='red', linewidths=0.5)
    plt.colorbar(label='ΔX (ppb km)')
    plt.title(f'Estimated rate {rate:.2f} t/h')
    plt.show()


## Example usage
Specify the point of interest and time range.

In [ ]:
lat = 31.6585lon = 5.9053start = '2020-07-01'end = '2020-07-31'column, mask, rate = analyze_scene(lat, lon, start, end)show_results(column, mask, rate)